In [1]:
import numpy as np
import pandas as pd
import os

from helper_classes import *
from utils import *
from similarity_predictor import check_similarity

C:\Glossary_Terms_Git\datahub-classify\venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
# ! pip install sentence_transformers

In [3]:
def get_public_data(input_data_path):
    print("===============%s=================" % input_data_path)
    data1 = pd.read_csv(os.path.join(input_data_path, "UCI_Credit_Card.csv"))
    data2 = pd.read_csv(os.path.join(input_data_path, "Age2_address1_credit_card3.csv"))
    data3 = pd.read_csv(os.path.join(input_data_path, "list_of_real_usa_addresses.csv"))
    data4 = pd.read_csv(os.path.join(input_data_path, "CardBase.csv"))
    data5 = pd.read_csv(os.path.join(input_data_path, "Credit_Card2.csv"))
    data6 = pd.read_csv(os.path.join(input_data_path, "catalog.csv"))
    data7 = pd.read_csv(os.path.join(input_data_path, "iban.csv"))
    data8 = pd.read_csv(os.path.join(input_data_path, "USA_cars_datasets.csv"))
    data9 = pd.read_csv(os.path.join(input_data_path, "email_1.csv"))
    data10 = pd.read_csv(os.path.join(input_data_path, "email_2.csv"))
    data11 = pd.read_csv(os.path.join(input_data_path, "email_3.csv"))
    data12 = pd.read_csv(
        os.path.join(input_data_path, "2018-seattle-business-districts.csv")
    )
    data13 = pd.read_csv(os.path.join(input_data_path, "Customer_Segmentation.csv"))
    data14 = pd.read_csv(os.path.join(input_data_path, "application_record.csv"))
    data15 = pd.read_csv(os.path.join(input_data_path, "Airbnb_Open_Data.csv"))
    data16 = pd.read_csv(
        os.path.join(input_data_path, "Book1.xlsx-credit-card-number.csv")
    )
    data17 = pd.read_csv(os.path.join(input_data_path, "Aliases.csv"))
    data18 = pd.read_csv(os.path.join(input_data_path, "athletes.csv"))
    data19 = pd.read_csv(os.path.join(input_data_path, "coaches.csv"))
    data20 = pd.read_csv(os.path.join(input_data_path, "curling_results.csv"))
    data21 = pd.read_csv(os.path.join(input_data_path, "Emails.csv"))
    data22 = pd.read_csv(os.path.join(input_data_path, "hockey_players_stats.csv"))
    data23 = pd.read_csv(os.path.join(input_data_path, "hockey_results.csv"))
    data24 = pd.read_csv(os.path.join(input_data_path, "medals.csv"))
    data25 = pd.read_csv(os.path.join(input_data_path, "Persons.csv"))
    data26 = pd.read_csv(os.path.join(input_data_path, "technical_officials.csv"))
    data27 = pd.read_csv(os.path.join(input_data_path, "Bachelor_Degree_Majors.csv"))
    data28 = pd.read_csv(os.path.join(input_data_path, "CrabAgePrediction.csv"))
    data29 = pd.read_csv(os.path.join(input_data_path, "Salary_Data.csv"))
    data30 = pd.read_csv(os.path.join(input_data_path, "drug-use-by-age.csv"))
    data31 = pd.read_csv(
        os.path.join(input_data_path, "Book1.xlsx-us-social-security-22-cvs.csv")
    )
    data32 = pd.read_csv(os.path.join(input_data_path, "sample-data.csv"))
    data33 = pd.read_excel(os.path.join(input_data_path, "1-MB-Test.xlsx"))
    data34 = pd.read_csv(os.path.join(input_data_path, "random_ibans.csv"))
    data35 = pd.read_csv(
        os.path.join(input_data_path, "used_cars_data.csv"), nrows=1000
    )
    data36 = pd.read_csv(os.path.join(input_data_path, "train.csv"), nrows=1000)
    data37 = pd.read_csv(os.path.join(input_data_path, "test.csv"), nrows=1000)
    data38 = pd.read_csv(os.path.join(input_data_path, "vehicles_1.csv"), nrows=1000)
    data39 = pd.read_csv(os.path.join(input_data_path, "vehicles_2.csv"), nrows=1000)
    data40 = pd.read_csv(os.path.join(input_data_path, "vehicles_3.csv"), nrows=1000)
    data41 = pd.read_csv(
        os.path.join(input_data_path, "Dataset-Unicauca-Version2-87Atts_1.csv")
    )
    data42 = pd.read_csv(
        os.path.join(input_data_path, "Dataset-Unicauca-Version2-87Atts_2.csv")
    )
    data43 = pd.read_csv(
        os.path.join(input_data_path, "Dataset-Unicauca-Version2-87Atts_3.csv")
    )
    data44 = pd.read_csv(
        os.path.join(input_data_path, "Dataset-Unicauca-Version2-87Atts_4.csv")
    )
    data45 = pd.read_csv(
        os.path.join(input_data_path, "Dataset-Unicauca-Version2-87Atts_5.csv")
    )
    data46 = pd.read_csv(
        os.path.join(input_data_path, "visitor-interests.csv"), nrows=1000
    )
    data47 = pd.read_csv(
        os.path.join(input_data_path, "Darknet_.csv"), nrows=1000, on_bad_lines="skip"
    )
    data48 = pd.read_csv(os.path.join(input_data_path, "vehicles_4.csv"))
    data49 = pd.read_csv(os.path.join(input_data_path, "vehicles_5.csv"))
    data50 = pd.read_csv(
        os.path.join(
            input_data_path, "Device Report - BU175-VPC2021-03-21_11-00-03.csv"
        )
    )
    data51 = pd.read_csv(
        os.path.join(
            input_data_path,
            "2021-04-23_honeypot-cloud-digitalocean-geo-1_netflow-extended.csv",
        ),
        nrows=1000,
    )
    data52 = pd.read_csv(os.path.join(input_data_path, "ipv6_random_generated.csv"))
    data53 = pd.read_csv(
        os.path.join(input_data_path, "score-banks-updated-sep2022.csv")
    )
    data54 = pd.read_csv(os.path.join(input_data_path, "blz-aktuell-xlsx-data.csv"))
    data55 = pd.read_csv(os.path.join(input_data_path, "automotive_service_data.csv"))
    data56 = pd.read_excel(os.path.join(input_data_path, "US_Driving_License.xlsx"))

    return {
        "data1": data1,
        "data2": data2,
        "data3": data3,
        "data4": data4,
        "data5": data5,
        "data6": data6,
        "data7": data7,
        "data8": data8,
        "data9": data9,
        "data10": data10,
        "data11": data11,
        "data12": data12,
        "data13": data13,
        "data14": data14,
        "data15": data15,
        "data16": data16,
        "data17": data17,
        "data18": data18,
        "data19": data19,
        "data20": data20,
        "data21": data21,
        "data22": data22,
        "data23": data23,
        "data24": data24,
        "data25": data25,
        "data26": data26,
        "data27": data27,
        "data28": data28,
        "data29": data29,
#         "data30": data30,
#         "data31": data31,
#         "data32": data32,
#         "data33": data33,
#         "data34": data34,
#         "data35": data35,
#         "data36": data36,
#         "data37": data37,
#         "data38": data38,
#         "data39": data39,
#         "data40": data40,
#         "data41": data41,
#         "data42": data42,
#         "data43": data43,
#         "data44": data44,
#         "data45": data45,
#         "data46": data46,
#         "data47": data47,
#         "data48": data48,
#         "data49": data49,
#         "data50": data50,
#         "data51": data51,
#         "data52": data52,
#         "data53": data53,
#         "data54": data54,
#         "data55": data55,
#         "data56": data56,
    }

In [4]:
dataset_name_mapping = {
        "data1": "UCI_Credit_Card",
        "data2": "Age2_address1_credit_card3",
        "data3": "list_of_real_usa_addresses",
        "data4": "CardBase",
        "data5": "Credit_Card2",
        "data6": "catalog",
        "data7": "iban",
        "data8": "USA_cars_datasets",
        "data9": "email_1",
        "data10": "email_2",
        "data11": "email_3",
        
        "data12": "2018-seattle-business-districts",
        "data13": "Customer_Segmentation",
        "data14": "application_record",
        "data15": "Airbnb_Open_Data",
        "data16": "Book1.xlsx-credit-card-number",
        "data17": "Aliases",
        "data18": "athletes",
        "data19": "coaches",
        "data20": "curling_results",
        "data21": "Emails",
        "data22": "hockey_players_stats",
        "data23": "hockey_results",
        "data24": "medals",
    
        "data25": "Persons",
        "data26": "technical_officials",
        "data27": "Bachelor_Degree_Majors",
        "data28": "CrabAgePrediction",
        "data29": "Salary_Data",
    }


platforms = ["A","B","C","D","E"]


In [5]:
input_data_dir = "C:\\Glossary_Terms_Git\\datahub-classify\\test\\datasets"
public_data_list = get_public_data(input_data_dir)

===============C:\Glossary_Terms_Git\datahub-classify\test\datasets=================


In [6]:

def populate_tableinfo_object(dataset_key):
    table_meta_info = {
            "Name": dataset_name_mapping[dataset_key],
            "Description": f"This table contains description of {dataset_name_mapping[dataset_key]}",
            "Platform": platforms[np.random.randint(0,5)],
            "Table_Id": dataset_key
                        }

    col_infos=[]
    for col in public_data_list[dataset_key].columns:
        fields = {
                    'Name': col,
                    'Description': f' {col}',
                    'Datatype': public_data_list[dataset_key][col].dropna().dtype ,
                    'Dataset_Name': dataset_key,
                    'Column_Id': dataset_key + "_" + col
                }
        metadata_col = Metadata(fields)
        parent_cols = []
        col_info = ColumnInfo(metadata_col, parent_cols)
        col_infos.append(col_info)

    metadata_table = Metadata_table(table_meta_info)
    parent_tables= []
    table_info = TableInfo(metadata_table, parent_tables, col_infos)
    return table_info

def populate_similar_tableinfo_object(dataset_key):
    df = public_data_list[dataset_key].copy()
    random_df_key = "data" + str(np.random.randint(1,len(dataset_name_mapping)))
    while not (dataset_name_mapping.get(random_df_key, None)) or (random_df_key == dataset_key):
        random_df_key = "data" + str(np.random.randint(1,len(dataset_name_mapping)))
    random_df = public_data_list[random_df_key].copy()
    
    df.columns = [ dataset_key + "_" + col for col in df.columns]
    random_df.columns = [random_df_key + "_" + col for col in random_df.columns]
    second_df = pd.concat([df, random_df], axis=1)
    cols_to_keep = list(df.columns) + list(random_df.columns[:2])
    second_df = second_df[cols_to_keep]
#     print(second_df.head())

    table_meta_info = {
        "Name": dataset_name_mapping[dataset_key] + "_v2",
        "Description": f" {dataset_name_mapping[dataset_key]}",
        "Platform": platforms[np.random.randint(0,5)],
        "Table_Id": dataset_key + "_" + random_df_key
                    }

    col_infos=[]
    for col in second_df.columns:
        fields = {
                    'Name': col.split("_",1)[1],
                    'Description': f'{col.split("_",1)[1]}',
                    'Datatype': second_df[col].dropna().dtype,
                    'Dataset_Name': dataset_key + "_" + random_df_key,
                    'Column_Id':  col
                }
        metadata_col = Metadata(fields)
        parent_cols = []
        col_info = ColumnInfo(metadata_col, parent_cols)
        col_infos.append(col_info)
    metadata_table = Metadata_table(table_meta_info)
    parent_tables= []
    table_info = TableInfo(metadata_table, parent_tables, col_infos)
    return table_info
    

In [7]:
# table_info_1 = populate_tableinfo_object(dataset_key = "data3")
# table_info_2 = populate_similar_tableinfo_object(dataset_key = "data3")
# overall_table_similarity_score, column_similarity_scores = check_similarity(table_info_1, table_info_2)
# overall_table_similarity_score,column_similarity_scores

In [8]:
# table_info_1 = populate_tableinfo_object(dataset_key = "data3")
# table_info_2 = populate_tableinfo_object(dataset_key = "data5")
# overall_table_similarity_score, column_similarity_scores = check_similarity(table_info_1, table_info_2)
# overall_table_similarity_score,column_similarity_scores

In [9]:
# table_info_1 = populate_tableinfo_object(dataset_key = "data2")
# table_info_2 = populate_tableinfo_object(dataset_key = "data27")
# overall_table_similarity_score, column_similarity_scores = check_similarity(table_info_1, table_info_2)
# overall_table_similarity_score,column_similarity_scores

In [10]:
table_info_1 = populate_tableinfo_object(dataset_key = "data9")
table_info_2 = populate_tableinfo_object(dataset_key = "data10")
overall_table_similarity_score, column_similarity_scores = check_similarity(table_info_1, table_info_2)
overall_table_similarity_score,column_similarity_scores

finding table similarity


C:\Glossary_stage_2\datahub-classify\datahub-classify\src\datahub_classify\utils.py:192: RuntimeWarning: Mean of empty slice.
  emb_1 = np.array(
C:\Glossary_Terms_Git\datahub-classify\venv\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Glossary_stage_2\datahub-classify\datahub-classify\src\datahub_classify\utils.py:194: RuntimeWarning: Mean of empty slice.
  emb_2 = np.array(


name score:  0.9999999999999999
desc score:  0.98
platforms:  A C
platform score:  1
lineae score:  0
schema score:  0.8571428571428571
********************finding column similarities**********************
total pairs -->  304


(1.0,
 {('data9_Unnamed: 0', 'data10_Unnamed: 0'): 1.0,
  ('data9_Unnamed: 0', 'data10_CODE'): 0.36,
  ('data9_Unnamed: 0', 'data10_NAME'): 0.4,
  ('data9_Unnamed: 0', 'data10_ADD1'): 0.07,
  ('data9_Unnamed: 0', 'data10_ADD2'): 0.07,
  ('data9_Unnamed: 0', 'data10_CITY'): 0.22,
  ('data9_Unnamed: 0', 'data10_STATE'): 0.31,
  ('data9_Unnamed: 0', 'data10_PIN'): 0.15,
  ('data9_Unnamed: 0', 'data10_TEL'): 0.12,
  ('data9_Unnamed: 0', 'data10_FAX'): 0.1,
  ('data9_Unnamed: 0', 'data10_E_MAIL'): 0.15,
  ('data9_Unnamed: 0', 'data10_DESIG1'): 0.23,
  ('data9_Unnamed: 0', 'data10_CEO_MD'): 0.24,
  ('data9_Unnamed: 0', 'data10_DESIG2'): 0.18,
  ('data9_Unnamed: 0', 'data10_G_MANAGER'): 0.12,
  ('data9_Unnamed: 0', 'data10_CAPACITY'): 0.26,
  ('data9_CODE', 'data10_Unnamed: 0'): 0.36,
  ('data9_CODE', 'data10_CODE'): 1.0,
  ('data9_CODE', 'data10_NAME'): 0.42,
  ('data9_CODE', 'data10_ADD1'): 0.28,
  ('data9_CODE', 'data10_ADD2'): 0.28,
  ('data9_CODE', 'data10_CITY'): 0.23,
  ('data9_CODE', 

In [11]:
a = ["sss"]
b = ["s","d"]

mean_ = np.mean(np.array([k for k in b if k in a ]))
print(mean_)
# print(len(mean_))
if str(mean_)!= "nan":
    print("::::::")

nan


C:\Glossary_Terms_Git\datahub-classify\venv\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [14]:
fuzzy_match_score = fuzz.ratio("city", "capacity") / 100
fuzzy_match_score

0.67